In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [ ]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [3]:
DROP TABLE IF EXISTS interacao;
CREATE VIEW interacao AS
    SELECT DU1.codeDrug A, DU2.codeDrug B
    FROM DrugUse DU1, DrugUse DU2 
    WHERE DU1.idPerson = DU2.idPerson AND DU1.codeDrug < DU2.codeDrug; 
/* eu realizei essa query utilando '<>' que é equivalente a '!=', no entanto essa query restornava o valor duplicado de cada tupla (A,B), ou seja
retornava (A,B) e (B,A), notei que o primeiro valor era sempre menor, então decidi pegar somente o primeiro valor que aparecia, (A,B).
*/

In [4]:
DROP TABLE IF EXISTS numeroInteracoes;
CREATE VIEW numeroInteracoes AS
    SELECT A, B, COUNT(*) numeroDeInteracoes_A_B
    FROM Interacao    
    GROUP BY A, B;        

In [ ]:
SELECT A,B, numeroDeInteracoes_A_B FROM numeroInteracoes
WHERE numeroDeInteracoes_A_B > 30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [ ]:
DROP TABLE IF EXISTS DrugInteraction;
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [ ]:
DROP TABLE IF EXISTS TotalInteracoes;
CREATE TABLE TotalInteracoes (
    Total DOUBLE NOT NULL,
    PRIMARY KEY(Total),
    )
    AS SELECT COUNT(druga) 
    FROM DrugInteraction;
    
SELECT * FROM totalInteracoes; /* Numero total de interações registradas 9073.0*/

SELECT DrugA, DrugB, ((weight)/(CAST (total AS DOUBLE))) 
AS Suporte 
FROM DrugInteraction, TotalInteracoes
WHERE weight > 30
ORDER BY Suporte;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [ ]:
CREATE VIEW GrupoMais30 AS 
SELECT * 
FROM DrugInteraction 
WHERE weight > 30;

In [ ]:
SELECT * FROM GrupoMais30;

### Explicação da Confiança:
A confiança seria calculada, utiliznado a soma do Weight de uma determinada tupla, dividido pela soma do weight de todas as vezes que o antedecendente aparecesse na tabela.
Por Exemplo, 96 -> 216, tem como weight 60, o weight total do 96 é 213, portanto a Confiação de 96 -> 216 é de 60/213 = 0.28169014084;

Não consegui executar uma query de maneira que conseguisse calcular a Confiança.

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.